# 🤺 Demo: Using the Environment in Agents and Tools

### 🤔 What is the environment?

When creating an agent, it is possible to pass an environment dictionary to the agent. This environment dictionary can be used to store information that is relevant to the agent, but not part of the model or tools. This information can be accessed by the agent and its delegates and tools during execution. This allows for the agent to be called with a specific set of hardcoded values that are scoped to that agent, e.g. a customer ID, a user ID, or a specific configuration. 

### ⚙️ How does it work?

The environment is passed to the agent when it is created. The agent can then access the environment using the `self.environment` attribute. The environment is a dictionary, so it can store any type of data that is relevant to the agent. The agent, by default, passes this environment to its tools when they are executed. This allows the tools to access the environment as well.

### 📄 Example: Using an orchestrator agent to pass a customer ID to a tool and a delegate agent

In [1]:
# Import the necessary classes
from fence.troupe.superagent import SuperAgent
from fence.tools.base import BaseTool
from fence.utils.logger import setup_logging
from fence.models.openai import GPT4omini
logger = setup_logging(__name__, log_level='info', serious_mode=False)

Let's build a basic tool that retrieves the account holder name from a database. The tool will use the `current_account_id` from the environment to determine which account holder name to return. The tool will return "Bert" if the `current_account_id` is "foo", "Ernie" if the `current_account_id` is "bar", and "Unknown" otherwise.

Remember, tools take their description from either the `description` attribute on init, or the docstring if no description is passed.

In [2]:
class AccountNameRetrieverTool(BaseTool):
    """
    Tool to retrieve the account holder name from a database.
    """
    def execute_tool(self, environment):
        account_id = self.environment.get("current_account_id", "unknown")
        logger.info(f"Retrieving account holder name for account_id: {account_id}")
        if account_id == "foo":
            return "Bert"
        if account_id == "bar":
            return "Ernie"
        return "Unknown"

Now let's build an agent that uses the `AccountNameRetrieverTool` to retrieve the account holder name. The agent will not explicitly need to pass anything to the tool, as the tool will access the `current_account_id` from the environment. The agent will set the `current_account_id` to "foo" and then run the tool to retrieve the account holder name.

In [3]:
# Create an agent with a model and tools
agent = SuperAgent(
    identifier='accountant',
    model=GPT4omini(source="agent"),
    tools=[AccountNameRetrieverTool()],
    environment={"current_account_id": "foo"},
)

agent.run('what is the current account holders name?')    # "How much is 9 + 10?",
    

[2024-09-29 09:55:04] [INFO] ℹ️ [superagent.__init__:77] Initialized agent with model <gpt-4o-mini>, delegates: [], tools: ['AccountNameRetrieverTool']
[2024-09-29 09:55:04] [INFO] ℹ️ [links.run:207] Executing <accountant_step_0> Link
[2024-09-29 09:55:05] [INFO] ℹ️ [superagent.run:114] Model response: [THOUGHT]I need to retrieve the account holder's name from the database since it is not something I can provide without specific data. I have a tool that can help me obtain this information. 
[ACTION]
```toml
tool_name = "AccountNameRetrieverTool"
```
[2024-09-29 09:55:05] [INFO] ℹ️ [superagent._handle_tool_action:198] Executing tool 'AccountNameRetrieverTool' with params: {}
[2024-09-29 09:55:05] [INFO] ℹ️ [2957405099.execute_tool:7] Retrieving account holder name for account_id: foo
[2024-09-29 09:55:05] [INFO] ℹ️ [superagent._handle_tool_action:203] Tool <AccountNameRetrieverTool> response: Bert
[2024-09-29 09:55:05] [INFO] ℹ️ [links.run:207] Executing <accountant_step_1> Link
[2024-0

"The current account holder's name is Bert."

Now let's create two agents, one delegating to the other, to verify whether the environment is passed to the delegate. The parent agent will set the `current_account_id` to "bar" and delegate to the child agent. The child agent will use the `AccountNameRetrieverTool` to retrieve the account holder name.

In [6]:
# Create the agents
child_agent = SuperAgent(
    identifier='child_accountant',
    description="Specialized accountant agent for account specifics.",
    model=GPT4omini(source="agent"),
    tools=[AccountNameRetrieverTool()],
)
parent_agent = SuperAgent(
    identifier='parent_accountant',
    model=GPT4omini(source="agent"),
    delegates=[child_agent],
    environment={"current_account_id": "bar"},
)
parent_agent.run('what is the current account holders name?')

[2024-09-29 09:59:08] [INFO] ℹ️ [superagent.__init__:77] Initialized agent with model <gpt-4o-mini>, delegates: [], tools: ['AccountNameRetrieverTool']
{}
[2024-09-29 09:59:08] [INFO] ℹ️ [superagent.__init__:77] Initialized agent with model <gpt-4o-mini>, delegates: ['child_accountant'], tools: []
{'current_account_id': 'bar'}
[2024-09-29 09:59:08] [INFO] ℹ️ [links.run:207] Executing <parent_accountant_step_0> Link
[2024-09-29 09:59:11] [INFO] ℹ️ [superagent.run:114] Model response: [THOUGHT]I do not have any information regarding account holders or their names. Since I have access to a delegate who specializes in account specifics, I will delegate this task to the child_accountant agent. 
[DELEGATE]
```toml
delegate_name = "child_accountant"
delegate_input = "Please provide the current account holder's name."
```
[2024-09-29 09:59:11] [INFO] ℹ️ [links.run:207] Executing <child_accountant_step_0> Link
[2024-09-29 09:59:12] [INFO] ℹ️ [superagent.run:114] Model response: [THOUGHT]I need 

"The current account holder's name is Ernie."